<font size=6>

<b>Curso de Programación en Python</b>
</font>

<font size=4>
    
Curso de formación interna, CIEMAT. <br/>
Madrid, marzo de 2023

Antonio Delgado Peris
</font>

https://github.com/andelpe/curso-intro-python/

<br/>

# Tema 8 - Otros recursos _(avanzados)_

## Objetivos

- Aprender a usar y gestionar las excepciones en Python
- Introducir/profundizar en algunos recursos sintácticos _avanzados_
  - _Comprenhensions_
  - Decoradores
  - Iteradores y generadores
  - Context managers 

## Excepciones

Permiten alterar el flujo de control de un programa, saltando la secuencia normal:

    caller -> f1 -> f2 -> result2 -> f1 -> result1 -> caller
    caller -> f1 -> f2 -> exception -> caller
    
Se suelen usar para controlar errores o situaciones excepcionales.

Con las sentencia `try` y `except` se pueden capturar excepciones (errores) provocadas por el código de nuestros programas, para que no resulten _fatales_ (no terminen el programa). 

El uso de `try/except` puede permitir evitar:

- La comprobación de errores tras cada operación:

```c                                          
if (doSth()!=0)    return ERROR
if (doMore()!=0)   return ERROR
if (whatever()!=0) return ERROR
```

```python
try:
    doSth()
    doMore()
except:
    return ...
```
          
- Comprobaciones previas:

```python
    if total > 0:               try:
        ratio = part/total          ratio = part/total
    else:                       except:
        ratio = None                ratio = None
```



Uso del bloque `try/except`:

```python
try:
    # Normal run code

except (Excp1, Excp2):
    # Execute if Excp1 or Excp2 were raised in try part

except Excp3 as ex:    
    # Execute if Excp3 (accessible as ‘ex’) was raised

except:
    # Execute if a different exception was raised

else:
    # Execute if no exception was raised

finally:
    # Execute no matter what (even if exception in catchers)
    # Still, propagate non-caught exceptions

```

La instancia `ex` contiene información adicional. Su interfaz mínimo es aceptar cualquier entrada (string) en su construcción.

- Lo muestra con el método: `__str__`. P. ej., con `print(ex)`

Un ejemplo:

In [ ]:
f = None

try:
    
    f = open('myfile.txt', 'w')
    print('-- File was opened\n')
    
    f.write(str(3/0))
    
except IOError as ex:
    print('ERROR: Dealing with file:', ex)
    
except ArithmeticError as ex:
    print('ERROR: Maths error:', ex)
    
except Exception as ex:
    print('ERROR: Unexpected:', ex)
    
finally:
    if f: f.close(); 
    print('\n-- File was closed')
    from pathlib import Path
    Path('myfile.txt').unlink()
    print('-- File was removed')

<br/>

Nota: Como vimos, el `finally` anterior puede (_y debería_) sustituirse por una sentencia `with`

<br/>

<div style="background-color:powderblue;">

**EJERCICIO e8_1:** 

Definir dos funciones `div` y `div2`, que acepten un string `a` y un número `b` y hagan lo siguiente:

- Convertir `a` a float. Si la conversión produce `ValueError`, se debe capturar e indicarlo por pantalla.
- Realizar la división `a/b` y, o bien mostrar el resultado, o bien un mensaje de error, si se produce un `ZeroDivisionError`.

La única diferencia entre `div` y `div2` es que la primera capturará explícitamente `ValueError` y `ZeroDivisionError` en un bloque `try/except`, mientras que la segunda solo puede capturar la genérica `Exception`.

Probarlo con el código siguiente:

In [ ]:
def div(a, b):  pass
def div2(a, b):  pass

for a, b in ('34', 0), ('X', 2), ('X', 0), ('34', 2):
    print('Input:', a, b)
    div(a, b)
    div2(a, b)
    print()

### Lanzar (y definir) excepciones

Las excepciones son objetos (_sorprendente_), que derivan de la clase base `Exception` (o de una derivada de ella).

- El árbol de herencia de las excepciones permite crear familias
- Una instrucción `except MiClase` captura excepciones de `MiClase`, o de cualquier clase derivada
- Es habitual que un proyecto Python defina su propia clase base derivada, y luego define tantas hijas como requiera

Por ejemplo:

    ArithmeticError --> OverflowError, ZeroDivisionError ...

La sentencia `raise` permite lanzar excepciones de la clase que se desee (o directamente `Exception`):

    raise Exception    # es igual que: raise Exception()
    raise Exception('Too big')

In [ ]:
print('Empezando...')
raise Exception('Stop here!')
print('...acabando.')

<br/>

<div style="background-color:powderblue;">

**EJERCICIO e8_2:** 

- Definir una función que acepte un entero y nos devuelva su representación binaria (`bin`)
- Si el argumento no es un valor entero debe lanzar una excepción de un tipo nuevo `NumeroError`, indicándolo.

## Comprenhensions
Una _list comprehension_ es una forma concisa de crear una lista usando un bucle implícito.

Su forma básica es:

```python
[<expr(x)> for x in <Iterable>]
```

In [ ]:
strings = ['this', 'and', 'those']

lengths = []
for s in strings:
    lengths.append(len(s))
print(lengths)

lengths2 = [len(s) for s in strings]
print(lengths2)

In [ ]:
matriz = [[0,1], [0,2]]
[x[0] for x in matriz]

La forma extendida de las _list comprenhensions_ es:

```python
[<expr(x,y)>  for x in I1   for y in I2   if <cond(x,y)>]
```

In [ ]:
v = (0, 1, 2)
[x  for x in v  if (x % 2) == 0]  

In [ ]:
w = (10, 16)
[x+y  for x in v  for y in w]

<br/>

<div style="background-color:powderblue;">

**EJERCICIO e8_3:** 

Dado el grafo `g`, representado por un diccionario, obtener el máximo número de conexiones de un nodo, en una sola línea.

In [ ]:
g = {
    'A': ['B', 'C'],
    'B': ['A', 'C', 'D'],
    'C': ['A'],
    'D': ['B', 'C'],
}

- Una _dict comprenhension_ es la forma análoga de crear diccionarios de manera concisa:

In [ ]:
{x: x**2 for x in (2, 4, 6)}

In [ ]:
{x: (2*i+2)**2  for i, x in enumerate(('a', 'b', 'c'))}

## Decoradores

Los decoradores son funciones que modifican a otras funciones. Equivalen a lo siguiente:

```python
myfunc = decorador(myfunc)

```

La función `decorador` acepta un objeto función como argumento, y devuelve un nuevo objeto función, tras realizar alguna modificación sobre ella.

Para facilitar su uso, los decoradores se pueden aplicar añadiendo una línea `@decorador` delante de la definición de una función. En el caso, anterior sería:

```python
@decorador
def myfunc(..):
    bla bla
```

Un ejemplo tonto sería:

In [ ]:
def hay_que_saludar(func):
    def wrapper(x):
        print('Hola Antonio!')
        return func(x)
    return wrapper

@hay_que_saludar
def double(x):
    return 2*x

print(double(5))

Un ejemplo con más utilidad potencial, de uno de los módulos de ejemplo:

In [ ]:
from modulos.decorador import timer

@timer
def sumador(v):
    return f'{sum(v):.2e}'

print("Resultado:", sumador(range(1000**2)))

<br/>

Además de los decoradores de funciones, también existen decoradores de métodos de clases (como los ejemplos `@classmethod` y `@staticmethod`, que ya vimos), e incluso de clases mismas (con funcionamiento análogo: el decorador modifica la clase sobre la que se usa).

## Iteradores
  
Vimos que un objeto _iterable_ puede recorrerse elemento a elemento. Técnicamente, un iterable debe devolver un objeto _iterador_.

- Un iterable es un objeto que implementa el método `__iter__`, que devuelve un iterador para sus elementos. Se invoca llamando a `iter(mi_iterable)`

- Un iterador debe implementar el método `__next__`, que devuelve el siguiente elemento. Se invoca llamando a `next(mi_iterador)`

- Cuando no quedan más elementos, el iterador lanza una excepción `StopIteration`.

In [ ]:
l = [2, 4]

# Get an iterator 'i' on the iterable 'l'
i = iter(l)

# Consume a couple of items
print(next(i))
print(next(i))
print(next(i))

In [ ]:
l = [2, 4, 6, 8]

# Get an iterator 'i' on the iterable 'l'
i = iter(l)

# Consume a couple of items
print(next(i))
print(next(i))

print('Antes del for')

# Consume the rest
for x in i:  print(x)
    


De hecho, el funcionamiento del bucle `for` para recorrer un iterable, es el siguiente:

- Obtiene un iterador del iterable
- Llama al método `next` repetidamente, hasta que recibe un `StopIteration`

El sistema también funciona para hacer un `for` sobre un iterador (si hacemos `iter(iterador)` recibimos el mismo iterador).

<br/>

<div style="background-color:powderblue;">

**EJERCICIO e8_4:** 

Crear una función `funcfor`, que replique el funcionamiento de `for`, sin usar la sentencia `for`.

In [ ]:
def funcfor(v, f):
    """
    Aplica la función 'f' sobre todos los elementos del iterable v
    """
    # TODO: rellenar la función

# Probarlo con:
funcfor(l, print)

## Generadores

Los generadores ofrecen un interfaz alternativo (más moderno) de recorrer series de elementos.

Las funciones o expresiones _generadores_ producen un _objeto generador_, que es un tipo de iterador, que es siempre perezoso.
 - Una función/expresión generador equivale a un iterable (produce un iterador)
 - Un objeto generador _es_ un iterador, pero siempre perezoso

Las funciones generador no implementan el método `__iter__`. Utilizan la sentencia `yield`.
- La función generador no incluyen ninguna sentencia `return`, pero debe incluir `yield`.
- La sentencia `yield` devuelve un valor, pero no acaba la función, sino que queda _parada_ hasta una nueva llamada.
- Cuando la función completa (ejecuta su última línea y sale), produce un `StopIteration`

In [ ]:
def gen():
    i = 0
    while i<6:
        yield i
        i += 1

# La función generador devuelve el objeto generador (iterador). 
# No hace nada más (no se ejecuta 'i = 0')
g = gen()

# El objeto generador soporta 'next': causa la ejecución de la función hasta 'yield'
print(next(g))

print('Antes del for')

# Como 'g' es un iterador, también puede ser iterado usando 'for'
for i in g:
    print(i)

De hecho, una forma más simple (y habitual) de hacer lo anterior sería la siguiente (igual que con un iterable):

In [ ]:
for i in gen():
    print(i)

<br/>

También existe las expresiones generador, que son análogas a las _list comprehensions_, pero devuelven un generador.

In [ ]:
squares = [x*x for x in range(1,5)]
print(squares)

In [ ]:
squares = (x*x for x in range(1,5))
print(squares)
print(type(squares))

for sq in squares:
    print(sq)

<br/>

<div style="background-color:powderblue;">

**EJERCICIO e8_5:** 

Crear un iterable que produzca el factorial de números enteros desde 0 hasta un máximo dado. Hacerlo usando el interfaz de iteradores y el de generadores.
    
Probar las dos implementaciones con el código siguiente:

In [ ]:
for n, fact in enumerate(ITERABLE_FACTORIAL(6)):
    print(f'{n:2} --> {fact:4}')

## Context Managers

Los _context managers_ permiten asociar acciones automáticas a la adquisición y liberación de un objeto.

- El ejemplo más típico es la gestión de un fichero, que garantiza que se cerrará aunque ocurra un error.
- Otro ejemplo es del objeto `Lock` del módulo `threading`, que asegura que se hace un `release` al acabar:

```python
some_lock = threading.Lock()
with some_lock:
    do_something
```

No vamos a entrar en muchos detalles, pero existen dos manera de programar nuevos context managers:

1. En una clase:

   - Definir los métodos especiales `__enter__` y `__exit__`, con lo que se debe hacer al adquirir y liberar el objeto.


2. Con una _función generador_

   - Aplicando el decorador `contextlib.contextmanager`
   - Incluyendo una expresión del siguiente tipo:
  
```python
try:
    act_on_acquisition
    yield resource
finally:
    clean_on_release
```